In [1]:
import os, json, re, logging, time
from os.path import join, getsize
from pwd import getpwuid

In [2]:
## Setup the logger for debugging
logger = logging.getLogger('dir-size')
logger.setLevel(logging.DEBUG)
## Create file handler which logs even debug messages
fh = logging.FileHandler('dir-size.log')
fh.setLevel(logging.DEBUG)
# create formatter and add it to the handlers
formatter = logging.Formatter('{"time": "%(asctime)s" ,"name": "%(name)s" ,"level": "%(levelname)s" ,"message": "%(message)s"}')
fh.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)
logger.info('Start dir-size log')

In [3]:
## Directory of python script
print(os.getcwd())

/mnt/c/GitHub/mortie23/dir-size


## Get Directory Sizes Recursive

In [4]:
def getDirSize(dirstart,fileInfoFlag,debugFlag):
    ## A couple of log lines to get timing of runs
    logger.info('START - getDirSize')
    logger.info(dirstart)
    dirstart_contract = re.sub('[^A-Za-z0-9]+', '', dirstart)
    print(dirstart_contract)
    i = 1
    for root, dirs, files in os.walk(dirstart):
        numsep = root.count(os.path.sep)
        jsonLine={}
        jsonLine["i"]=i
        jsonLine["numsep"]=numsep
        jsonLine["directory"]=root
        fileset = []
        ## Get the file info as well
        if fileInfoFlag:
            for name in files:
                uid = os.stat(join(root, name))[5]
                st_mtime = os.stat(join(root, name))[9]
                fileset.append({"name":name, "size":getsize(join(root, name)), "uid": uid, "st_mtime": time.ctime(st_mtime)})
        jsonLine["fileset"]=fileset
        bytes_size=0
        ## Work around 32bit Python windows does not like really long paths
        for name in files:
            if len(join(root, name)) > 260:
                bytes_size += 0
            else:
                bytes_size += getsize(join(root, name))
        # test
        jsonLine["bytes"]=bytes_size
        jsonLine["files"]=len(files)
        if debugFlag:
            print(jsonLine)
        else:
            ## Create new file
            fProc = open("./json-data/" +  dirstart_contract + '.' + str(i) + '.json', "w")
            fProc.write(json.dumps(jsonLine))
            fCsv = open("./csvs/" + dirstart_contract + '.csv', "a+")
            csvLine = ','.join([str(i),dirstart_contract,"./json-data/" + dirstart_contract + '.' + str(i) + '.json'+"\n"])
            fCsv.write(csvLine)
            fProc.close()
            fCsv.close()
        i += 1
    ## A couple of log lines to get timing of runs
    logger.info('END - getDirSize')
    logger.info(dirstart)

In [5]:
## Unit test on small directory structure
## Default is to start at this directory
dirstart = os.getcwd()
getDirSize(dirstart,1,0)

mntcGitHubmortie23dirsize


In [7]:
## Section Admin
dirstart = "/mnt/c/Users/chris/Documents"
#getDirSize(dirstart,1,0)

# Loop Files
A function to loop over all files in a directory and call the getDirSize function

In [8]:
def loopFiles(dirname,debugFlag):
    directory = os.fsencode(dirname)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if debugFlag:
            print(filename)
            getDirSize(dirname+'/'+filename,1,1)
        else:
            getDirSize(dirname+'/'+filename,1,0)
            pass

In [9]:
## Unit test for a directory
dirname = "/mnt/c/Users/chris/"
#loopFiles(dirname,0)